In [93]:
import pandas as pd 
import numpy as np

In [94]:
file = "Purchase_data.csv"

In [95]:
purchase_data = pd.read_csv(file)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [96]:
# PLAYER COUNT
# Total number of players - Calculate this out by the SN column, there are repeats so make sure to only count them once 

total_players = len(purchase_data["SN"].value_counts())

total_players_dict = [{"Total": total_players}]

total_players_df = pd.DataFrame(total_players_dict)
total_players_df



,Total
0,576


In [97]:
# PURCHASING ANALYSIS 

# Calculate the number of unique items
items_unique = len(purchase_data["Item Name"].value_counts())

# Calculate the average price
avg_price = purchase_data["Price"].mean()
avg_price_round = round(avg_price,2)

# Calculate Number of Purchases
purchase_number = len(purchase_data["Item Name"])

# Calculate Total Revenue
total_rev = purchase_data["Price"].sum()

# Create a data frame for these findings 
purchasing_analysis = [{"Number of Unique Items": items_unique, "Average Price": avg_price_round, "Number of Purchases": purchase_number, "Total Revenue": total_rev}]

purchase_analysis_df = pd.DataFrame(purchasing_analysis)
purchase_analysis_df.style.format({"Average Price": "$ {:.2f}", "Total Revenue": "$ {:.2f}"})


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$ 3.05,780,$ 2379.77


In [98]:
# GENDER DEMOGRAPHICS 

# Counts - Break out the two columns needed, take out the duplicates to avoid errors in count, group by gender to get count
gender_original = purchase_data.loc[:,["SN", "Gender"]]
gender_df = gender_original.drop_duplicates(["SN"])
gender_df = gender_df.groupby("Gender")
gender_df = gender_df.count()


# Percentage - add a new column to the data frame, calculate the percentage, rename the SN column to Count (Credit: I looked at Dave's .map, .format code and used it here)
gender_df["Percentage of Players"] = (gender_df["SN"] / total_players*100).map("{:.2f}%".format)
gender_df = gender_df.rename(columns={"SN": "Count"})
gender_df



,Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [99]:
# PURCHASING ANALYSIS (Gender)

# Find the Purchase Count and create a data frame 
gender_purchase_df = gender_original.groupby("Gender")
gender_purchase_df = gender_purchase_df.count()
gender_purchase_df = gender_purchase_df.rename(columns={"SN": "Purchase Count"})

# Groupby the price and gender to add to data frame
gender_price = purchase_data.loc[:, ["Gender", "Price"]]
gender_price = gender_price.groupby("Gender")

# Find the average purchase price and place into data frame 
gender_purchase_df["Average Purchase Price"] = round(gender_price["Price"].mean(),2)

# Find the Total Purchase Value and place into data frame 
gender_purchase_df["Total Purchase Value"] = round(gender_price["Price"].sum(),2)

# # Find the Average Total Purchase per person 
gender_purchase_df["Average Total Purchase Per Person"] = round((gender_price["Price"].sum() / gender_df["Count"]),2)

gender_purchase_df.style.format({"Average Purchase Price": "$ {:.2f}", "Total Purchase Value": "$ {:.2f}", "Average Total Purchase Per Person": "$ {:.2f}"})



,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Gender,,,,
Female,113,$ 3.20,$ 361.94,$ 4.47
Male,652,$ 3.02,$ 1967.64,$ 4.07
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56


In [100]:
# AGE DEMOGRAPHICS 

# Bin the ages and add to the original dataframe
age_bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,100]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

pd.cut(purchase_data["Age"], age_bins, labels=age_labels)
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=age_labels)

# Create a data frame dividing the count by age group. 
age_original = purchase_data.loc[:,["SN", "Age Group"]]
age_df = age_original.drop_duplicates(["SN"])
age_df = age_df.groupby("Age Group")
age_df = age_df.count()
age_df = age_df.rename(columns={"SN": "Total Count"})

# Calculate percentage 
age_df["Percentage of Players"] = (age_df["Total Count"] / total_players*100).map("{:.2f}%".format)
age_df


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [101]:
# AGE PURCHASE ANALYSIS 

# Bin the ages and add to the original dataframe
age_bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,100]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

pd.cut(purchase_data["Age"], age_bins, labels=age_labels)
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=age_labels)

# Create a data frame with the column for purchase count 
age_purchase_original = purchase_data.loc[:,["SN", "Age Group"]]
age_purchase_df = age_purchase_original.groupby("Age Group")
age_purchase_df = age_purchase_df.count()
age_purchase_df = age_purchase_df.rename(columns={"SN": "Purchase Count"})

# Groupby age and price to add to data frame
age_price = purchase_data.loc[:, ["Age Group", "Price"]]
age_price = age_price.groupby("Age Group")

# Find the average purchase price and place into data frame 
age_purchase_df["Average Purchase Price"] = round(age_price["Price"].mean(),2)

# Find the Total Purchase Value and place into data frame 
age_purchase_df["Total Purchase Value"] = round(age_price["Price"].sum(),2)

# # Find the Average Total Purchase per person 
age_purchase_df["Average Total Purchase Per Person"] = round((age_price["Price"].sum() / age_df["Total Count"]),2)

age_purchase_df.style.format({"Average Purchase Price": "$ {:.2f}", "Total Purchase Value": "$ {:.2f}", "Average Total Purchase Per Person": "$ {:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Age Group,,,,
<10,23,$ 3.35,$ 77.13,$ 4.54
10-14,28,$ 2.96,$ 82.78,$ 3.76
15-19,136,$ 3.04,$ 412.89,$ 3.86
20-24,365,$ 3.05,$ 1114.06,$ 4.32
25-29,101,$ 2.90,$ 293.00,$ 3.81
30-34,73,$ 2.93,$ 214.00,$ 4.12
35-39,41,$ 3.60,$ 147.67,$ 4.76
40+,13,$ 2.94,$ 38.24,$ 3.19


In [102]:
# TOP SPENDERS 

# Create a Top Spenders data frame
top_original = purchase_data.loc[:,["SN", "Item ID"]]
top_original = top_original.groupby("SN")
top_original = top_original.count()
top_original = top_original.rename(columns={"Item ID": "Purchase Count"})

# Pull out the price and SN from the original data to make calculations
top_spend_price = purchase_data.loc[:, ["SN", "Price"]]
top_spend_price = top_spend_price.groupby("SN")

# Find the Average purchase price 
top_original["Average Purchase Price"] = round(top_spend_price["Price"].mean(),2)

# Find Total Purchase Value 
top_original["Total Purchase Value"] = round(top_spend_price["Price"].sum(),2)

top_spend_df = top_original.sort_values("Total Purchase Value", ascending=False)

top_spend_df.head(5).style.format({"Average Purchase Price": "$ {:.2f}", "Total Purchase Value": "$ {:.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$ 3.79,$ 18.96
Idastidru52,4,$ 3.86,$ 15.45
Chamjask73,3,$ 4.61,$ 13.83
Iral74,4,$ 3.40,$ 13.62
Iskadarya95,3,$ 4.37,$ 13.10


In [250]:
# MOST POPULAR ITEMS 

most_popular = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]

most_popular["Purchase Count"] = most_popular.groupby(['Item ID', 'Item Name'])["Price"].transform('count')
most_popular["Total Purchase Value"] = round((most_popular["Price"]*most_popular["Purchase Count"]),2)
most_popular = most_popular.sort_values("Purchase Count", ascending=False)
most_popular = most_popular.drop_duplicates(["Item ID"])

most_popular.head(5).style.format({"Price": "$ {:.2f}", "Total Purchase Value": "$ {:.2f}"})






,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
722,92,Final Critic,$ 4.88,13,$ 63.44
62,178,"Oathbreaker, Last Hope of the Breaking Storm",$ 4.23,12,$ 50.76
504,82,Nirvana,$ 4.90,9,$ 44.10
56,108,"Extraction, Quickblade Of Trembling Hands",$ 3.53,9,$ 31.77
538,132,Persuasion,$ 3.19,9,$ 28.71


In [249]:
# 
most_popular = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]
most_popular = most_popular.groupby(["Item ID", "Item Name"])
most_popular = most_popular.count()
most_popular = most_popular.rename(columns={"Price": "Purchase Count"})


popular_price = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]
popular_price = popular_price.groupby(["Item ID", "Item Name"])

most_popular["Item Price"] = round(popular_price["Price"].mean(),2)

most_popular["Total Purchase Value"] = round((most_popular["Purchase Count"]*most_popular["Item Price"]),2)

most_popular = most_popular.sort_values("Total Purchase Value", ascending=False)

most_popular.head(5).style.format({"Item Price": "$ {:.2f}", "Total Purchase Value": "$ {:.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$ 4.61,$ 59.93
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
82,Nirvana,9,$ 4.90,$ 44.10
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
103,Singed Scalpel,8,$ 4.35,$ 34.80
